In [4]:
# Broker_vendor.py
import pandas as pd
import xml.etree.ElementTree as ET
import psycopg2
from datetime import datetime
import re
import cx_Oracle

username = 'apps'
password = 'apps'
host = 'csebsd2db.cswg.com' #EBSD2
port = '1521'
service_name = 'csebsd2_int' #EBSD2

# Database connection parameters
db_params = {
    "host": "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com",
    "database": "CSPODB",
    "port": 5432,
    "user": "cspoms",
    "password": "cspoms",
    "sslmode": "verify-ca",
    "sslcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-cert.pem",
    "sslkey": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-key.pem",
    "sslrootcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\server-ca.pem"
}

post_conn = psycopg2.connect(**db_params)
post_cur = post_conn.cursor()

# Create a connection
dsn = cx_Oracle.makedsn(host, port, service_name=service_name)
oracle_con = cx_Oracle.connect(username, password, dsn)
oracle_cur = oracle_con.cursor()

In [5]:

post_cur.execute(f"SELECT * FROM XXPO_ORCL2CSPOMS_VEND_STG where PROCESS_STATUS ='N' LIMIT 1005")
# Fetch all rows into a list of tuples
result_rows = post_cur.fetchall()
# Get the column names from the cursor description
column_names = [desc[0] for desc in post_cur.description]
# Create a DataFrame using the result and column names
stg_df = pd.DataFrame(result_rows, columns=column_names)
# Print or use the DataFrame as needed
print('Loaded to staging table records "N" to stg_df')
# Extract the 'bic_vendor_number' column and convert it to a list
bic_vendor_numbers_list = stg_df['bic_vendor_number'].tolist()
# Convert the list of strings to a list of integers
bic_vendor_numbers_list = list(map(int, bic_vendor_numbers_list))
# # Print or use the list as needed
# print('bic_vendor_numbers_list--->', bic_vendor_numbers_list)
# # Convert the list of numbers to a string with quotes for the SQL query
# vendor_numbers_str = ', '.join([f"'{num}'" for num in bic_vendor_numbers_list])
# # Construct the SQL query
# sql_query = f"SELECT * FROM APPS.XXAP_VENDOR_MASTER WHERE VENDOR_NBR IN ({vendor_numbers_str})"
# oracle_df = pd.read_sql(sql_query, oracle_con)

Loaded to staging table records "N" to stg_df


In [9]:
len(bic_vendor_numbers_list)

1005

In [ ]:
# Construct the SQL query
sql_query = f"SELECT * FROM APPS.XXAP_VENDOR_MASTER WHERE VENDOR_NBR IN ({vendor_numbers_str})"
oracle_df = pd.read_sql(sql_query, oracle_con)
oracle_df

In [ ]:
vendor_numbers_str

In [ ]:
chunk_size = 1000
chunks = [bic_vendor_numbers_list[i:i + chunk_size] for i in range(0, len(bic_vendor_numbers_list), chunk_size)]
vendor_numbers_str = ', '.join([f"'{num}'" for num in chunks])

vendor_numbers_str

In [ ]:
result_df = pd.DataFrame()

for chunk in chunks:
    sql_query = f"SELECT * FROM APPS.XXAP_VENDOR_MASTER WHERE VENDOR_NBR IN ({vendor_numbers_str})"
    chunk_df = pd.read_sql(sql_query, oracle_con, params=[chunk])
    result_df = pd.concat([result_df, chunk_df], ignore_index=True)

In [7]:
chunk_size = 1000
chunks = [bic_vendor_numbers_list[i:i + chunk_size] for i in range(0, len(bic_vendor_numbers_list), chunk_size)]
chunks
# Create a list of strings for each chunk
chunk_strings = [', '.join([f"'{num}'" for num in chunk]) for chunk in chunks]
chunk_strings
# # Concatenate the chunk strings
vendor_numbers_str = ', '.join(chunk_strings)
vendor_numbers_str
# Now vendor_numbers_str will contain separate strings for each num within each chunk


"'8571', '8571', '8571', '8571', '8571', '8571', '8571', '8571', '8571', '8571', '8571', '8571', '8571', '8571', '8571', '2130', '2601', '278560', '31314', '34529', '34531', '34556', '34718', '34824', '34841', '403179', '60184', '60497', '61226', '61376', '101290', '25158', '25169', '25466', '25964', '26072', '693822', '693827', '445932', '693844', '693883', '693896', '693827', '693883', '61142', '8571', '445932', '445932', '445932', '693896', '25541', '25362', '693821', '445932', '445932', '447709', '447709', '447709', '447709', '693823', '693824', '693830', '693831', '37430', '417384', '447709', '8571', '447709', '8571', '693832', '693835', '693836', '693837', '693838', '693839', '693843', '693845', '693846', '693849', '693850', '693855', '693859', '693861', '693863', '693864', '693865', '693868', '693872', '693873', '693884', '61376', '693886', '693887', '693888', '693890', '693891', '693892', '693895', '693898', '693899', '693900', '693901', '693905', '693909', '693910', '693911', 

In [8]:
result_df = pd.DataFrame()

for chunk in chunks:
    # Create a string for each chunk
    chunk_str = ', '.join([f"'{num}'" for num in chunk])
    
    # Construct the SQL query using the chunk string
    sql_query = f"SELECT * FROM APPS.XXAP_VENDOR_MASTER WHERE VENDOR_NBR IN ({chunk_str})"
    
    # Read data for the current chunk and concatenate to result_df
    chunk_df = pd.read_sql(sql_query, oracle_con)
    result_df = pd.concat([result_df, chunk_df], ignore_index=True)

# Now result_df contains the concatenated data from all chunks
result_df

,VENDOR_SITE_ID,VENDOR_NBR,VENDOR_NAME,VENDOR_START_DATE,VENDOR_END_DATE,VENDOR_STATUS,VENDOR_TYPE,AP_VENDOR_ID,AP_VENDOR_NAME,AP_VENDOR_ALT_NAME,...,EOQ_CAP_DAYS,WHSE_SHIP_TO,SHIP_WITH_VENDOR,RANK_PERCENT_OVERRIDE,PO_DATE_CHANGE_RSN_FLAG,PULLBKS_AVAILABLE,HAWAII_TRANS_METHOD,TEST_NEW,VENDOR_BUYER_NBR,SHELVING_DELAY
0,1026264,100208,REVOLYMER US INC,2016-07-09 08:42:40,2017-06-06 00:00:00,I,Buyer Vendor,8839,REVOLYMER,REVOLYMER,...,None,None,None,None,None,None,None,None,129,None
1,1877017,100208,TROPICAL FRUIT & NUT CO,2018-01-20 10:43:32,2021-01-19 00:00:00,I,Buyer Vendor,5803,TROPICAL NUT AND FRUIT CO,TROPICAL NUT AND FRUIT NC,...,None,None,None,None,None,None,None,None,078,None
2,1026699,101290,NUK USA,2016-07-09 08:48:51,2018-01-27 10:23:37,I,Buyer Vendor,7156,NUK USA LLC,NUK USA LLC,...,0,0,000000,0,None,None,None,None,791,None
3,3274177,101290,None,2022-06-30 16:34:07,NaT,A,Buyer Vendor,1917056,NEWELL BRANDS DISTRIBUTION LLC,None,...,None,None,None,None,None,None,None,None,None,None
4,1887025,101290,NUK USA,2018-01-27 10:23:37,2022-06-30 16:34:07,I,Buyer Vendor,1295006,GRACO CHILDRENS PRODUCTS,None,...,0,0,000000,0,None,None,None,None,669,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1188,375051,693745,VENTURA FOODS. LLC,2015-04-25 05:57:31,None,A,Buyer Vendor,277,VENTURA FOODS,VENTURA FOODS,...,None,None,None,None,None,None,None,None,854,None
1189,375069,693761,FRESH GOURMET CO,2015-04-25 05:57:45,None,A,Buyer Vendor,1211,SUGAR FOODS CORP,SUGAR FOODS CORP,...,None,None,None,None,None,None,None,None,836,None
1190,375086,693762,UNISTAR PLASTICS. LLC,2015-04-25 05:57:59,None,A,Buyer Vendor,1264,UNISTAR PLASTICS LLC,UNISTAR PLASTICS LLC,...,None,None,None,None,None,None,None,None,836,None
1191,375070,693765,SALES. USA. INC,2015-04-25 05:57:46,None,A,Buyer Vendor,1564,SALES USA INC,SALES USA INC,...,None,None,None,None,None,None,None,None,836,None


In [ ]:
chunk_size = 1000
chunks = [bic_vendor_numbers_list[i:i + chunk_size] for i in range(0, len(bic_vendor_numbers_list), chunk_size)]
# chunks
# Create a list of strings for each chunk
chunk_strings = [', '.join([f"'{num}'" for num in chunk]) for chunk in chunks]
# chunk_strings
# # Concatenate the chunk strings
vendor_numbers_str = ', '.join(chunk_strings)
# vendor_numbers_str
# Now vendor_numbers_str will contain separate strings for each num within each chunk
result_df = pd.DataFrame()

for chunk in chunks:
    # Create a string for each chunk
    chunk_str = ', '.join([f"'{num}'" for num in chunk])
    
    # Construct the SQL query using the chunk string
    sql_query = f"SELECT * FROM APPS.XXAP_VENDOR_MASTER WHERE VENDOR_NBR IN ({chunk_str})"
    
    # Read data for the current chunk and concatenate to result_df
    chunk_df = pd.read_sql(sql_query, oracle_con)
    result_df = pd.concat([result_df, chunk_df], ignore_index=True)